## Data Prep -- XML to CSV

In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
health_date = ET.parse('export.xml')

In [3]:
root = health_date.getroot()
root

<Element 'HealthData' at 0x122d398f0>

In [4]:
root.tag

'HealthData'

In [5]:
root.attrib

{'locale': 'en_CN'}

In [ ]:
## look at a sample of the child nodes

i = 0
for child in root:
    print(child.tag)
    print(child.attrib)
    #print(child.attrib.get("type"))
    #print(child.attrib.get("sourceName"))
    i += 1
    if i >= 20:
        break

In [79]:
data_type_dic = {}

for child in root:
    if child.tag not in data_type_dic:
        data_type_dic[child.tag] = 0
    else:
        data_type_dic[child.tag] += 1

In [80]:
print(data_type_dic)

{'ExportDate': 0, 'Me': 0, 'Record': 627410, 'Workout': 159, 'ActivitySummary': 183}


In [89]:
### parse the three type of records into three pandas df

records_cols = ["type", "sourceName", "unit", "creationDate", "startDate", "endDate", "value"]
records_rows = []

workout_cols = ["type", "duration", "durationUnit", "totalDistance", "totalDistanceUnit", 
                "totalEnergyBurned", "totalEnergyBurnedUnit", "sourceName",
                "creationDate", "startDate", "endDate"]
workout_rows = []

summary_cols = ["date", "activeEnergyBurned", "activeEnergyBurnedGoal", "activeEnergyBurnedUnit",
               "exerciseTime", "exerciseTimeGoal", "standHours", "standHoursGoal"]
summary_rows = []

for node in root: 
    if node.tag == 'Record':
        r_type = node.attrib.get("type")
        r_sourceName = node.attrib.get("sourceName")
        r_unit = node.attrib.get("unit")
        r_creationDate = node.attrib.get("creationDate")
        r_startDate = node.attrib.get("startDate")
        r_endDate = node.attrib.get("endDate")
        r_value = node.attrib.get("value")

        records_rows.append({"type": r_type, "sourceName": r_sourceName, 
                             "unit": r_unit, "creationDate": r_creationDate,
                             "startDate": r_startDate, "endDate": r_endDate,
                             "value": r_value
                            })
    
    elif node.tag == 'Workout':
        w_type = node.attrib.get("workoutActivityType")
        w_duration = node.attrib.get("duration")
        w_durationUnit = node.attrib.get("durationUnit")
        w_totalDistance = node.attrib.get("totalDistance")
        w_totalDistanceUnit = node.attrib.get("totalDistanceUnit")
        w_totalEnergyBurned = node.attrib.get("totalEnergyBurned")
        w_totalEnergyBurnedUnit = node.attrib.get("totalEnergyBurnedUnit")
        w_sourceName = node.attrib.get("sourceName")
        w_unit = node.attrib.get("gunit")
        w_creationDate = node.attrib.get("creationDate")
        w_startDate = node.attrib.get("startDate")
        w_endDate = node.attrib.get("endDate")

        workout_rows.append({"type": w_type, 
                             "duration": w_duration, "durationUnit": w_durationUnit, 
                             "totalDistance": w_totalDistance, "totalDistanceUnit": w_totalDistanceUnit,
                             "totalEnergyBurned": w_totalEnergyBurned, "totalEnergyBurnedUnit": w_totalEnergyBurnedUnit,
                             "sourceName": r_sourceName, "creationDate": r_creationDate,
                             "startDate": r_startDate, "endDate": r_endDate
                            })
    
    elif node.tag == 'ActivitySummary':
        s_date = node.attrib.get("dateComponents")
        s_activeEnergyBurned = node.attrib.get("activeEnergyBurned")
        s_activeEnergyBurnedGoal = node.attrib.get("activeEnergyBurnedGoal")
        s_activeEnergyBurnedUnit = node.attrib.get("activeEnergyBurnedUnit")
        s_appleExerciseTime = node.attrib.get("appleExerciseTime")
        s_appleExerciseTimeGoal = node.attrib.get("appleExerciseGoal")
        s_appleStandHours = node.attrib.get("appleStandHours")
        s_appleStandHoursGoal = node.attrib.get("appleStandHoursGoal")

        summary_rows.append({"date": s_date,
                            "activeEnergyBurned": s_activeEnergyBurned,
                            "activeEnergyBurnedGoal": s_activeEnergyBurnedGoal,
                            "activeEnergyBurnedUnit": s_activeEnergyBurnedUnit,
                            "exerciseTime": s_appleExerciseTime, "exerciseTimeGoal": s_appleExerciseTimeGoal,
                            "standHours": s_appleStandHours, "standHoursGoal": s_appleStandHoursGoal})

record_df = pd.DataFrame(records_rows, columns = records_cols)
workout_df = pd.DataFrame(workout_rows, columns = workout_cols)
summary_df = pd.DataFrame(summary_rows, columns = summary_cols)


In [ ]:
record_df.head(10)

In [93]:
record_df['type'].value_counts()

HKQuantityTypeIdentifierActiveEnergyBurned                  242019
HKQuantityTypeIdentifierBasalEnergyBurned                   101057
HKQuantityTypeIdentifierStepCount                            85004
HKQuantityTypeIdentifierHeartRate                            84482
HKQuantityTypeIdentifierDistanceWalkingRunning               81194
HKQuantityTypeIdentifierAppleExerciseTime                     6541
HKQuantityTypeIdentifierFlightsClimbed                        6480
HKQuantityTypeIdentifierAppleStandTime                        5943
HKQuantityTypeIdentifierEnvironmentalAudioExposure            5850
HKCategoryTypeIdentifierAppleStandHour                        4149
HKCategoryTypeIdentifierSleepAnalysis                         1573
HKQuantityTypeIdentifierHeadphoneAudioExposure                1123
HKQuantityTypeIdentifierHeartRateVariabilitySDNN               827
HKCategoryTypeIdentifierMenstrualFlow                          519
HKQuantityTypeIdentifierRestingHeartRate                      

In [94]:
record_df.to_csv('all_health_record.csv', index=False)

In [95]:
heart_rate_df = record_df.loc[record_df['type'].isin(["HKQuantityTypeIdentifierHeartRate", 
                                                      "HKQuantityTypeIdentifierRestingHeartRate", 
                                                      "HKQuantityTypeIdentifierWalkingHeartRateAverage"])]
heart_rate_df.to_csv('heart_rate.csv', index=False)

In [96]:
heart_rate_df.head(10)

,type,sourceName,unit,creationDate,startDate,endDate,value
110,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 19:45:48 -0800,2019-08-29 19:45:14 -0800,2019-08-29 19:45:14 -0800,93
111,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 19:50:33 -0800,2019-08-29 19:47:20 -0800,2019-08-29 19:47:20 -0800,89
112,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 19:56:00 -0800,2019-08-29 19:53:11 -0800,2019-08-29 19:53:11 -0800,87
113,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 20:02:50 -0800,2019-08-29 19:57:39 -0800,2019-08-29 19:57:39 -0800,78
114,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 20:07:01 -0800,2019-08-29 20:03:24 -0800,2019-08-29 20:03:24 -0800,84
115,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 20:11:53 -0800,2019-08-29 20:05:47 -0800,2019-08-29 20:05:47 -0800,82
116,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 21:34:31 -0800,2019-08-29 20:10:58 -0800,2019-08-29 20:10:58 -0800,79
117,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 21:34:37 -0800,2019-08-29 21:34:36 -0800,2019-08-29 21:34:36 -0800,86
118,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 21:34:37 -0800,2019-08-29 21:34:37 -0800,2019-08-29 21:34:37 -0800,84
119,HKQuantityTypeIdentifierHeartRate,Yu’s Apple Watch,count/min,2019-08-29 21:34:39 -0800,2019-08-29 21:34:38 -0800,2019-08-29 21:34:38 -0800,85


In [97]:
energy_df = record_df.loc[record_df['type'].isin(["HKQuantityTypeIdentifierActiveEnergyBurned", 
                                                  "HKQuantityTypeIdentifierBasalEnergyBurned"])]
energy_df.to_csv('energy.csv', index=False)

In [98]:
energy_df.head(10)

,type,sourceName,unit,creationDate,startDate,endDate,value
250892,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-01-13 17:56:37 -0800,2019-01-13 17:34:50 -0800,2019-01-13 17:49:58 -0800,14.45
250893,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-01-13 18:09:18 -0800,2019-01-13 17:49:58 -0800,2019-01-13 18:04:57 -0800,15.375
250894,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-01-13 18:04:57 -0800,2019-08-30 07:25:27 -0800,9546.56
250895,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 07:25:27 -0800,2019-08-30 07:40:23 -0800,16.512
250896,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 07:40:23 -0800,2019-08-30 07:55:22 -0800,15.372
250897,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 07:55:22 -0800,2019-08-30 08:10:20 -0800,16.325
250898,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 08:10:20 -0800,2019-08-30 08:25:29 -0800,16.162
250899,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 08:25:29 -0800,2019-08-30 08:25:49 -0800,0.39
250900,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 08:25:49 -0800,2019-08-30 09:25:49 -0800,52.732
250901,HKQuantityTypeIdentifierBasalEnergyBurned,Yu’s Apple Watch,kcal,2019-08-30 12:09:15 -0800,2019-08-30 09:25:49 -0800,2019-08-30 09:50:23 -0800,21.583


In [99]:
activity_df = record_df.loc[record_df['type'].isin(["HKQuantityTypeIdentifierStepCount",
                                                    "HKQuantityTypeIdentifierFlightsClimbed", 
                                                    "HKQuantityTypeIdentifierDistanceWalkingRunning"])]
activity_df.to_csv('activity.csv', index=False)

In [100]:
activity_df.head(10)

,type,sourceName,unit,creationDate,startDate,endDate,value
84694,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 03:38:28 -0800,2015-02-17 03:38:31 -0800,14
84695,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 04:12:29 -0800,2015-02-17 04:12:31 -0800,12
84696,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 04:12:44 -0800,2015-02-17 04:12:46 -0800,6
84697,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 04:12:46 -0800,2015-02-17 04:12:49 -0800,1
84698,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 08:05:00 -0800,2015-02-17 08:05:03 -0800,6
84699,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 08:05:03 -0800,2015-02-17 08:05:05 -0800,4
84700,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 08:05:10 -0800,2015-02-17 08:05:13 -0800,7
84701,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 17:39:47 -0800,2015-02-17 17:39:52 -0800,3
84702,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 17:39:57 -0800,2015-02-17 17:40:02 -0800,14
84703,HKQuantityTypeIdentifierStepCount,iPhone,count,2015-02-17 19:16:08 -0800,2015-02-17 18:18:30 -0800,2015-02-17 18:18:33 -0800,10


In [101]:
active_time_df = record_df.loc[record_df['type'].isin(["HKQuantityTypeIdentifierAppleExerciseTime", 
                                                       "HKQuantityTypeIdentifierAppleStandTime"])]
active_time_df.to_csv('active_time.csv', index=False)

In [102]:
active_time_df.head(10)

,type,sourceName,unit,creationDate,startDate,endDate,value
600448,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 07:36:42 -0800,2019-08-30 07:34:54 -0800,2019-08-30 07:35:54 -0800,1
600449,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 07:37:46 -0800,2019-08-30 07:35:54 -0800,2019-08-30 07:36:54 -0800,1
600450,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 07:38:27 -0800,2019-08-30 07:37:19 -0800,2019-08-30 07:38:19 -0800,1
600451,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 07:39:31 -0800,2019-08-30 07:38:19 -0800,2019-08-30 07:39:19 -0800,1
600452,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 07:46:04 -0800,2019-08-30 07:41:58 -0800,2019-08-30 07:42:58 -0800,1
600453,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-30 08:11:22 -0800,2019-08-30 08:07:09 -0800,2019-08-30 08:08:09 -0800,1
600454,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-31 10:40:42 -0800,2019-08-31 10:36:11 -0800,2019-08-31 10:37:11 -0800,1
600455,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-31 10:40:42 -0800,2019-08-31 10:37:11 -0800,2019-08-31 10:38:11 -0800,1
600456,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-31 12:43:12 -0800,2019-08-31 12:41:24 -0800,2019-08-31 12:42:24 -0800,1
600457,HKQuantityTypeIdentifierAppleExerciseTime,Yu’s Apple Watch,min,2019-08-31 13:52:40 -0800,2019-08-31 13:51:32 -0800,2019-08-31 13:52:32 -0800,1


In [91]:
workout_df.head(10)

,type,duration,durationUnit,totalDistance,totalDistanceUnit,totalEnergyBurned,totalEnergyBurnedUnit,sourceName,creationDate,startDate,endDate
0,HKWorkoutActivityTypeTraditionalStrengthTraining,2.038662616411845,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
1,HKWorkoutActivityTypeTraditionalStrengthTraining,16.9852432012558,min,0,km,112,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
2,HKWorkoutActivityTypeTraditionalStrengthTraining,2.130942714214325,min,0,km,10,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
3,HKWorkoutActivityTypeTraditionalStrengthTraining,2.085643335183462,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
4,HKWorkoutActivityTypeTraditionalStrengthTraining,1.923131616910299,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
5,HKWorkoutActivityTypeTraditionalStrengthTraining,2.026056496302287,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
6,HKWorkoutActivityTypeTraditionalStrengthTraining,2.031076717376709,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
7,HKWorkoutActivityTypeTraditionalStrengthTraining,2.129126334190369,min,0,km,10,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
8,HKWorkoutActivityTypeTraditionalStrengthTraining,2.089483785629272,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800
9,HKWorkoutActivityTypeTraditionalStrengthTraining,1.930689167976379,min,0,km,9,kcal,Yu’s Apple Watch,2019-12-21 01:04:21 -0800,2019-12-21 01:01:30 -0800,2019-12-21 01:04:20 -0800


In [103]:
workout_df.to_csv('workout.csv', index=False)

In [92]:
summary_df.head(10)

,date,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,exerciseTime,exerciseTimeGoal,standHours,standHoursGoal
0,1969-12-30,0,0,kcal,0,None,0,12
1,1969-12-31,0,0,kcal,0,None,0,12
2,2019-01-12,0,0,kcal,0,None,0,12
3,2019-01-13,2.289,0,kcal,0,None,0,12
4,2019-08-28,0,0,kcal,0,None,0,12
5,2019-08-29,0,270,kcal,0,None,2,12
6,2019-08-30,227.007,270,kcal,6,None,12,12
7,2019-08-31,282.434,270,kcal,8,None,13,12
8,2019-09-01,314.599,500,kcal,7,None,14,12
9,2019-09-02,168.857,350,kcal,0,None,11,12


In [104]:
summary_df.to_csv('daily_summary.csv', index=False)